## Qualtiy of neural control reflects local gradient alignment

Here we perform three experiments:
1. Gradient alignment vs. standard linear regression EV as a function of sample size
2. Original strech/OHP control objective vs. gradient alignment EV from linear mapping
3. New vector control objective vs. gradient alignment EV. 

The goal of #1 is to see whether better pointwise function alignment leads to better gradient alignment.
If this is not the case, then this would explan #2. Given these observations we improve upon the control metric's sensitivity to gradient with #3.

In [1]:
import torch
import torchvision.models as models
from util.modelregressions import CNNCrossFit
from util.misc_functions import float2rgb
from dataset.hvm import HVMDataset
import matplotlib.pyplot as plt
import numpy as np
import os

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cuda


In [ ]:
# load up target and source models
squeezenet_target = models.squeezenet1_0(pretrained=True).features.to(device)
squeezenet_source = models.squeezenet1_1(pretrained=True).features.to(device)
# define basic params 
cnn_layer = 12
source_units = np.arange(512)
target_units = np.arange(512)
img_dim = [256,256,3]
# pre-load data into memory for speed
nimg = 10;
batch_max = 1;
hvmdataset = HVMDataset(device,nimg)

C:\Users\Chong Guo\AppData\Local\conda\conda\envs\torch\lib\site-packages\torchvision\models\squeezenet.py:94: UserWarning: nn.init.kaiming_uniform is now deprecated in favor of nn.init.kaiming_uniform_.
  init.kaiming_uniform(m.weight.data)
C:\Users\Chong Guo\AppData\Local\conda\conda\envs\torch\lib\site-packages\torchvision\models\squeezenet.py:92: UserWarning: nn.init.normal is now deprecated in favor of nn.init.normal_.
  init.normal(m.weight.data, mean=0.0, std=0.01)


In [ ]:
# create a control experiment (squeezenet1_1-->squeezenet1_0)
snet2snet = CNNCrossFit(squeezenet_target,squeezenet_source,cnn_layer,target_units,source_units,device)
# extract layer activation across two models
snet2snet.design(hvmdataset, batch_size = batch_max, record_gradient = True)

Extracting activations and input jacobians


  1%|▊                                                                             | 52/5000 [05:28<8:41:54,  6.33s/it]

In [ ]:
snet2snet200.X

In [ ]:
# linear prediction of target unit, record performance
snet2snet.fit(8,2)
# score the test set
snet2snet.score()
# control units
snet2snet.control()

In [ ]:
plt.figure(figsize=(14,3))
plt.subplot(1,3,1)
plt.plot(snet2snet.train_score,snet2snet.test_gscore,'.')
plt.plot([0,1],[0,1],'k--')
plt.xlim([0,1])
plt.ylim((-5,1))
plt.xlabel('train score')
plt.ylabel('test gradient score')
plt.subplot(1,3,2)
plt.plot(snet2snet.test_gscore,snet2snet.source_act,'.')
plt.legend()
plt.axis('tight')
plt.xlim((-5,1))
plt.ylim((lmin-5,lmax/2))
plt.xlabel('test gradient score')
plt.ylabel('optimized act. source model')
plt.subplot(1,3,3)
plt.plot(snet2snet.test_gscore,snet2snet.ctr_score,'.')
plt.legend()
plt.xlim((-5,1))
plt.ylim((0,1))
plt.xlabel('test grad score')
plt.ylabel('fraction act. source/target')
plt.show()